<a href="https://colab.research.google.com/github/PTson2207/Deployment-Model-App/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training someone model, then deploy ...

- Want: Save_model and then upload Google Storage
- Data: From Kaggle-Food 101
- Model(s): 10, 11, 12 classes

Setup some Function

In [1]:
# thư viện
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Hàm unzip dowload_data file

def unzip_data(file_name):
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [3]:
# Setup dataset input
IMG_SIZE = (224, 224)

def create_data_loader(train_dir, test_dir, img_size=IMG_SIZE):
    train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                     label_mode="categorical",
                                                                     image_size=img_size)
    
    test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                    label_mode="categorical",
                                                                    image_size=img_size)
    return train_data, test_data

In [4]:
data_augument = keras.Sequential([
                                  preprocessing.RandomFlip('horizontal'),
                                  preprocessing.RandomRotation(0.2),
                                  preprocessing.RandomHeight(0.2),
                                  preprocessing.RandomWidth(0.2),
                                  preprocessing.RandomZoom(0.2),
                                  #preprocessing.Rescaling(1./255) # cho Resnet50V2, không cần thiết với EfficientNetB0
], name="data_augument")

In [5]:
# setup input_shape and base_model
INPUT_SHAPE = (244, 244, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

16711680/16705208 [==============================] - 0s 0us/step


In [6]:
def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
    # Fine tune model
    base_model.trainable = False
    # tạo layers
    inputs = layers.Input(shape=INPUT_SHAPE, name="input_layer")
    # thêm data_augument như một layer
    x = data_augument(inputs)
    # give model inputs (after augument) nhưng không train
    x = base_model(x, training= False)
    # Pool layer
    x = layers.GlobalAveragePooling2D(name='pooling_layers')(x)
    # put a dense layer as a out_put
    outputs = layers.Dense(num_classes, activation='softmax', name='output_layers')(x)

    # model từ input và output
    model = keras.Model(inputs, outputs)

    model.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    model.summary()

    return model

In [7]:
# Hàm import ảnh và resize ảnh
def load_and_prep_img(filename, img_shape=244, scale=False):
    # đọc ảnh
    img = tf.io.read_file(file_name)
    # dịch ảnh sang tensor unit8
    img = tf.image.decode_jpeg(img)
    # thay đổi kích thước ảnh
    img = tf.image.resize(img, [img_shape, img_shape])
    # đổi giá trị ảnh về giữa đoạn 0 đến 1
    if scale:
        return img/255
    else:
        return img

Download Data

In [8]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

unzip_data("10_food_classes_all_data.zip")

--2021-03-19 09:56:06--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M   210MB/s    in 2.4s    

2021-03-19 09:56:09 (210 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



In [9]:
# có bao nhiêu ảnh trong mỗi folder ?
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_all_data"):
    print(f"Có {len(dirnames)} thư mục con và có {len(filenames)} ảnh trong thư mục '{dirpath}'.")

Có 2 thư mục con và có 0 ảnh trong thư mục '10_food_classes_all_data'.
Có 10 thư mục con và có 0 ảnh trong thư mục '10_food_classes_all_data/train'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/pizza'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/fried_rice'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/sushi'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/hamburger'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/steak'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/grilled_salmon'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/ice_cream'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/chicken_curry'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10_food_classes_all_data/train/chicken_wings'.
Có 0 thư mục con và có 750 ảnh trong thư mục '10

In [18]:
# hàm tạo nhằm mục đích load lại mỗi khi chạy mới mô hình
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
                log_dir=log_dir
    )
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

    Vì ta có khả năng có thể chạy lại model nhiều lần, vì thế ta có thể theo dõi chúng theo một cách nào đó.
    Hàm trên lưu lại nhật kí hiệu suất mô hình ở thư mục: [dir_name]/[experiment_name]/[current_timestamp]:
    - dir_name:  thư mục lưu lại nhật kí tổng thể.
    - experiment_name: thử nghiệm cụ thể.
    - current_timestamp: thời gian thử nghiệm chi tiết.

Model 1 (10 classes)

In [11]:
train_data, test_data = create_data_loader(train_dir="10_food_classes_all_data/train/",
                                           test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [12]:
# xem kích cỡ dataset
train_data

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [13]:
# call model
model_1 = create_model(num_classes=len(train_data.class_names))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 244, 244, 3)]     0         
_________________________________________________________________
data_augument (Sequential)   (None, None, None, 3)     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
pooling_layers (GlobalAverag (None, 1280)              0         
_________________________________________________________________
output_layers (Dense)        (None, 10)                12810     
Total params: 4,062,381
Trainable params: 12,810
Non-trainable params: 4,049,571
_________________________________________________________________


In [19]:
history_1_percent = model_1.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)), # validate for less steps
                    # Track model training logs
                    callbacks=[create_tensorboard_callback("transfer_learning", "all_data_aug")])

Saving TensorBoard log files to: transfer_learning/all_data_aug/20210319-100713
Epoch 1/5


ValueError: ignored